In [1]:
import geopandas as gpd
import pandas as pd
import requests
import os
import shutil
import getpass
import glob
import json
import duckdb
import io

In [2]:
http_proxy = getpass.getpass(prompt='http_proxy')
https_proxy = getpass.getpass(prompt='https_proxy')
proxies = { "http"  : http_proxy, "https" : https_proxy}

http_proxy ········
https_proxy ········


In [3]:
s = requests.get('https://www.data.gouv.fr/fr/datasets/r/721bdff1-14b0-4fa5-a949-bd12f9ce714b', proxies=proxies).text

liste_file = pd.read_json(io.StringIO(s))

In [4]:
liste_file_93 = liste_file[liste_file.proj=='lamb93']
liste_file_93.head()

,id,name,url,proj,dep,year
2,4583734b-1a2d-4630-8975-7e2c497befaa,ocs-ge-2-0-shp-lamb93-d095-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d095,2021
3,3cc771a8-dfa5-4fb3-9cf1-f1c6f7b0576b,ocs-ge-2-0-shp-lamb93-d095-2018-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d095,2018
4,d79d16d0-bd15-4a04-8e25-6abcac552da2,ocs-ge-2-0-shp-lamb93-d094-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d094,2021
5,3a9b6089-b4c7-4697-b942-ad38152b2601,ocs-ge-2-0-shp-lamb93-d094-2018-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d094,2018
6,a674e8db-9cb8-4d33-9e4e-552b7def1c21,ocs-ge-2-0-shp-lamb93-d093-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d093,2021


In [5]:
url = liste_file_93.url.iloc[0]

In [9]:
duckdb.sql("SET http_proxy = '"+ http_proxy + "';")

In [21]:
duckdb.sql('INSTALL SPATIAL;LOAD SPATIAL;')

In [30]:
duckdb.sql("SELECT * FROM '{}' LIMIT 1".format(url)) 

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────┬───────────┬─────────┬───────────┬─────────┬──────────┬────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│            ID            │  CODE_CS  │ CODE_US │ MILLESIME │ SOURCE  │ OSSATURE │ ID_ORIGINE │ CODE_OR │                                                                              geometry                                                                               │
│         varchar          │  varchar  │ varchar │  varchar  │ varchar │  int32   │  varchar   │ varchar │                                                                              geometry                                                                               │
├──────────────────────────┼───────────┼─────────┼───────────┼─────────┼──────────┼────────────┼─────────┼───────────────────────────────────────────────────────────────────────────

In [31]:
duckdb.sql("SELECT CODE_CS, sum(st_area(geometry)) FROM '{}' group by CODE_CS".format(url)) 

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────┬────────────────────────┐
│  CODE_CS  │ sum(st_area(geometry)) │
│  varchar  │         double         │
├───────────┼────────────────────────┤
│ CS2.2.1   │      709811008.1073476 │
│ CS1.1.1.2 │      91032799.03120703 │
│ CS2.1.1.1 │     315039216.66984725 │
│ CS2.1.2   │     15196201.312700137 │
│ CS1.1.1.1 │      94789447.16554657 │
│ CS2.1.1.2 │       2272920.53015035 │
│ CS2.1.3   │     29765.911000016407 │
│ CS1.1.2.1 │     14041188.368150303 │
│ CS1.2.2   │      9844322.512449471 │
│ CS2.1.1.3 │     1484888.8495997686 │
│ CS1.2.1   │     15949.176649975225 │
│ CS1.1.2.2 │      8146.834249978114 │
├───────────┴────────────────────────┤
│ 12 rows                  2 columns │
└────────────────────────────────────┘